In [1]:
import os
import json
import numpy as np
import torch as pt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.signal import bessel, sosfiltfilt

import transport_signal_processing as tsp
from data_manager import DataManager

In [2]:
# parameters
output_dir = "runtime_data"

# make folders
os.makedirs(output_dir, exist_ok=True)

In [3]:
# parameters
path = "*100mV*"
level = 1
selected_only = True

# setup database connector
sigman = DataManager('database')

# load segments informations
sinfo_l = sigman.load_info(path, 's*')

# convert info to dataframe
df = pd.DataFrame(sinfo_l)

# condition key
exp_keys = ['pore', 'voltage', 'analyte']
df['condition'] = df.apply(lambda x: '-'.join([str(x[k]) for k in exp_keys]), axis=1)

# experience id
exp_keys = ['pore', 'analyte', 'voltage', 'temperature', 'channel', 'replica']
df['uid'] = df.apply(lambda x: '-'.join([str(x[k]) for k in exp_keys]), axis=1)

# manual filter
#df = df[df['analyte'].isin(['mix15taupY18WT', 'mix11taupY18WT', 'mixWTpS129', 'mix20taupY18WT'])]

# keep only selected segments
df = df[(df['selected'] > (level-1))]
df

,pore,analyte,voltage,temperature,channel,replica,part,sid,segment_range,segment_duration,MODIFIED,mI_open,sI_open,N_events,N_cores,N_reduced,selected,ratio_sel,condition,uid
16,K238A,psFLasynGluC,100,25,004,20,000,2,"[27984142, 89999999]",620.15857,2022-10-05_17:24:43,59.298965,1.909643,1062,333.0,333.0,1.0,0.993994,K238A-100-psFLasynGluC,K238A-psFLasynGluC-100-25-004-20
17,K238A,psFLasynGluC,100,25,004,20,000,0,"[0, 9284736]",92.84736,2022-10-05_17:24:43,57.720828,2.040034,138,52.0,52.0,1.0,0.961538,K238A-100-psFLasynGluC,K238A-psFLasynGluC-100-25-004-20
18,K238A,psFLasynGluC,100,25,004,20,000,1,"[11248177, 26896059]",156.47882,2022-10-05_17:24:43,58.854694,1.886648,324,90.0,90.0,1.0,0.977778,K238A-100-psFLasynGluC,K238A-psFLasynGluC-100-25-004-20
29,K238A,psFLasynGluC,100,25,004,21,003,0,"[0, 90099999]",900.99999,2022-10-05_17:24:43,69.689750,2.171546,2420,713.0,713.0,1.0,0.992987,K238A-100-psFLasynGluC,K238A-psFLasynGluC-100-25-004-21
30,K238A,psFLasynGluC,100,25,002,7,001,4,"[30057867, 30945792]",8.87925,2022-10-05_17:24:43,93.536089,2.600489,22,4.0,4.0,1.0,1.000000,K238A-100-psFLasynGluC,K238A-psFLasynGluC-100-25-002-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,K238A,psAYEMPSEE,100,25,003,04,000,1,"[1451721, 63987318]",625.35597,2022-10-05_17:24:42,73.243465,1.627581,59575,23989.0,23989.0,1.0,0.999500,K238A-100-psAYEMPSEE,K238A-psAYEMPSEE-100-25-003-04
174,K238A,psAYEMPSEE,100,25,001,09,000,0,"[0, 1714579]",17.14579,2022-10-05_17:24:42,67.170532,2.152668,766,342.0,342.0,1.0,1.000000,K238A-100-psAYEMPSEE,K238A-psAYEMPSEE-100-25-001-09
176,K238A,psAYEMPSEE,100,25,001,09,000,1,"[1784763, 37255711]",354.70948,2022-10-05_17:24:42,67.296616,2.057267,17263,8115.0,8115.0,1.0,0.999754,K238A-100-psAYEMPSEE,K238A-psAYEMPSEE-100-25-001-09
179,K238A,psAYEMPSEE,100,25,001,01,000,0,"[0, 249836543]",2498.36543,2022-10-05_17:24:42,66.748912,2.616005,113705,63653.0,63653.0,1.0,0.999733,K238A-100-psAYEMPSEE,K238A-psAYEMPSEE-100-25-001-01


In [4]:
df['analyte'].unique()

array(['psFLasynGluC', 'GAPQEGILE', 'AYEMPSEE', 'GYQDYEPEA', 'DMPVDPDNE',
       'FLasynGluC', 'MPSEE', 'psAYEMPSEE'], dtype=object)

In [5]:
def remove_tails(y, sigma_tol=0.0):
    y_thr = np.mean(y) + sigma_tol * np.std(y)
    i0, i1 = np.where(y < y_thr)[0][[0,-1]]
    return y[i0:i1]

def events_to_features(events, min_output_size=16, sigma_tol=0.0):
    X_l = []
    for evt in events:
        if len(evt) > 1:
            #y = evt[:,1]
            #y = remove_tails(y, sigma_tol=sigma_tol)
            y = evt.copy()
            y[:,0] = y[:,0] - y[0,0]
            if len(y) > min_output_size:
                X_l.append(y)
    return X_l

In [6]:
# paramters
N_min = 50

# find keys
ckey_sels = np.unique(df["condition"].values)

# extract features and labels from events
data = {}
for k, ckey in enumerate(ckey_sels):
    # condition subset
    dfs = df[df["condition"] == ckey].copy()
    
    # group by experiment id
    ukey_sels = np.unique(dfs["uid"].values)
    for ukey in tqdm(ukey_sels):
        dfu = dfs[dfs["uid"] == ukey]
        
        # load core events
        events = tsp.utils.load_core_events(sigman, dfu.to_dict('records'), selected_only=selected_only)

        # extract features
        x_l = events_to_features(events)
        
        # store data
        if len(x_l) > N_min:
            # stats fix
            #mx = np.mean([np.mean(x) for x in x_l])
            #sx = np.mean([np.std(x) for x in x_l])
            #x_l = [(x - mx)/sx for x in x_l]

            data[ukey] = x_l
        else:
            print(ukey, len(x_l))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 38.78it/s]


In [7]:
# pack results
for key in data:
    X_l = data[key]
    np.savez(
        os.path.join(output_dir, key+".npz"),
        X=np.array(X_l, dtype=np.dtype('object')),
    )